In [2]:
import pandas as pd

In [22]:
def inspect_csv(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    print("done reading csv")

    # Display column header names
    print("Column headers:")
    print(df.columns)

    # Display the first 5 rows
    print("\nFirst 5 rows:")
    print(df.head())

    # Display the last 5 rows
    print("\nLast 5 rows:")
    print(df.tail())

    # Display the number of rows and columns
    print("\nNumber of rows and columns:")
    print(df.shape)

In [20]:
import dask.dataframe as dd

def load_hmda_corrected(file_path):
    # Define explicit dtypes for problematic columns
    dtype = {
        'applicant_race_name_2': 'object',
        'applicant_race_name_3': 'object',
        'applicant_race_name_4': 'object',
        'applicant_race_name_5': 'object',
        'co_applicant_race_name_2': 'object',
        'co_applicant_race_name_3': 'object',
        'co_applicant_race_name_4': 'object',
        'co_applicant_race_name_5': 'object',
        'county_code': 'float64',  # Allow for NA
        'denial_reason_name_2': 'object',
        'denial_reason_name_3': 'object',
        'hud_median_family_income': 'float64',  # Allow for NA
        'loan_amount_000s': 'float64',  # Allow for NA
        'msamd': 'float64',  # Allow for NA
        'number_of_1_to_4_family_units': 'float64',  # Allow for NA
        'number_of_owner_occupied_units': 'float64',  # Allow for NA
        'population': 'float64',  # Allow for NA
        'state_code': 'float64',  # Allow for NA
    }

    # Load the CSV with the specified dtypes
    df = dd.read_csv(file_path, dtype=dtype)

    # Display column headers
    print("Column headers:")
    print(df.columns)

    # Display the first 5 rows
    print("\nFirst 5 rows:")
    print(df.head(5))

    # Display the last 5 rows
    print("\nLast 5 rows:")
    print(df.tail(5))  # No need for .compute()

    # Display the number of rows and columns
    print("\nNumber of rows and columns:")
    rows, cols = df.shape
    print(f"Rows: {rows.compute()}, Columns: {cols}")


In [21]:
load_hmda_corrected('../HDMA/hmda_2017_nationwide_all-records_labels.csv')

Column headers:
Index(['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr',
       'agency_code', 'loan_type_name', 'loan_type', 'property_type_name',
       'property_type', 'loan_purpose_name', 'loan_purpose',
       'owner_occupancy_name', 'owner_occupancy', 'loan_amount_000s',
       'preapproval_name', 'preapproval', 'action_taken_name', 'action_taken',
       'msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code',
       'county_name', 'county_code', 'census_tract_number',
       'applicant_ethnicity_name', 'applicant_ethnicity',
       'co_applicant_ethnicity_name', 'co_applicant_ethnicity',
       'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2',
       'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3',
       'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5',
       'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1',
       'co_applicant_race_name_2', 'co_applicant_race_2',
       'co_appl

/Users/rodri/Desktop/MIT Fall 2024 Classes/6.3950/Project/myenv/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


Rows: 14285496, Columns: 78


In [23]:
# inspect hmda_2017_panel.csv

inspect_csv('../HDMA/hmda_2017_panel.csv')

done reading csv
Column headers:
Index(['Activity Year', 'Respondent ID', 'Agency Code', 'Parent Respondent ID',
       'Parent Name (Panel)', 'Parent City (Panel)', 'Parent State (Panel)',
       'Region', 'Assets', 'Other Lender Code', 'Respondent Name (Panel)',
       'Respondent City (Panel)', 'Respondent State (Panel)',
       'Top Holder RSSD ID', 'Top Holder Name', 'Top Holder City',
       'Top Holder State', 'Top Holder Country', 'Respondent RSSD ID',
       'Parent RSSD ID', 'Respondent FIPS State Number'],
      dtype='object')

First 5 rows:
   Activity Year Respondent ID  Agency Code  Parent Respondent ID  \
0           2017    0002736291            9             2138466.0   
1           2017    0003783948            9             3226762.0   
2           2017    0000722777            9             3981856.0   
3           2017    0000212465            9             1378434.0   
4           2017    0003212149            9             2967509.0   

                  Parent 

In [24]:
# hmda_2017_transmittal_sheet.csv

inspect_csv('../HDMA/hmda_2017_transmittal_sheet.csv')

done reading csv
Column headers:
Index(['Activity Year', ' Respondent ID', ' Agency Code', ' Federal Tax ID',
       ' Respondent Name', ' Respondent Mailing Address', ' Respondent City',
       ' Respondent State', ' Respondent ZIP Code', ' Parent Name',
       ' Parent Address', ' Parent City', ' Parent State', ' Parent ZIP Code',
       'LAR Count'],
      dtype='object')

First 5 rows:
   Activity Year  Respondent ID   Agency Code  Federal Tax ID  \
0           2017     0000011286             3      59-0238640   
1           2017     0000033542             3      87-0489793   
2           2017     0000001821             3      35-0318380   
3           2017     0000004122             5      46-0242124   
4           2017     0000058874             3      26-2218160   

                 Respondent Name  Respondent Mailing Address  Respondent City  \
0       Farmers & Merchants Bank    200 E. Washington Street       Monticello   
1               ROCK CANYON BANK            215 W 2230